<a href="https://colab.research.google.com/github/AnanyaSharma2/Deep-Learning-Projects/blob/main/Projects/Credit%20Risk%20Analysis/CreditRiskAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')
import os

## **Loading the Dataset**

In [ ]:
bank = pd.read_excel("/content/case_study2.xlsx")
cibil= pd.read_excel("/content/case_study1.xlsx")

In [ ]:
df_b = bank.copy()
df_c = cibil.copy()

In [ ]:
bank.sample(2)

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
19321,19322,199,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,-99999.000,0,0,ConsumerLoan,ConsumerLoan,672,P2
30879,30880,89,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,1.656,0,0,ConsumerLoan,others,676,P2


In [ ]:
cibil.sample(2)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
6638,6639,3,3,0,0,2,0.000,0.667,0.000,1.000,...,0,2,1,0,0,1,2,0,20,7
11725,11726,7,1,6,1,0,0.143,0.000,0.857,0.143,...,0,2,1,0,0,3,4,2,60,6


In [ ]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    51336 non-null  int64  
 1   time_since_recent_payment     51336 non-null  int64  
 2   time_since_first_deliquency   51336 non-null  int64  
 3   time_since_recent_deliquency  51336 non-null  int64  
 4   num_times_delinquent          51336 non-null  int64  
 5   max_delinquency_level         51336 non-null  int64  
 6   max_recent_level_of_deliq     51336 non-null  int64  
 7   num_deliq_6mts                51336 non-null  int64  
 8   num_deliq_12mts               51336 non-null  int64  
 9   num_deliq_6_12mts             51336 non-null  int64  
 10  max_deliq_6mts                51336 non-null  int64  
 11  max_deliq_12mts               51336 non-null  int64  
 12  num_times_30p_dpd             51336 non-null  int64  
 13  n

In [ ]:
cibil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

# **Data Wrangling**

In [ ]:
df_c['Age_Oldest_TL'].unique()

array([    72,      7,     47,      5,    131,    150,     17,     36,
           16,     66,     64,     96,     49,     38,      9,      6,
          110,    138,      8,     92,     40,     11,     51,     59,
           37,    159,     10,     20,      4,     26,     19,     41,
           73,     45,     32,     33,     48,     18,     60,     14,
           83,     44,     24,     42,     39,     12,     27,     70,
           76,    120,    115,     46,     93,     56,     61,    113,
           67,     74,     22,    191,      3,     65,    192,     43,
           13,     29,    193,     98,     63,     58,     30,     23,
           69,     53,    145,     31,     77,    104,     87,     15,
           62,     21,     97,     34,     28,    137,     86,    124,
          129,     50,     35,      2,    102,    154,    148,    128,
           94,    107,    135,     81,     68,     78,    130,     91,
           71,     89,    123,    213,     88,     52,    175,      1,
      

As we can see that -99999 is a odd value so removing now removing this value from both the dataset

# **Removing the values from cibil dataset copy**

In [ ]:
df_c = df_c.loc[df_c['Age_Oldest_TL']!= -99999]

Original dataset contain 51336 rows

In [ ]:
cibil.shape

(51336, 26)

After removing the odd values we have 51296 rows

In [ ]:
df_c.shape

(51296, 26)

# **Removing the odd values from the bank dataset**

Method for Handling Odd Values:

This approach combines two strategies to deal with odd values (potentially representing outliers or errors) in a dataset:

1. Column-Wise Analysis and Removal

Frequency Calculation: For each column, calculate the frequency of odd values .
Threshold-Based Removal: If the count of odd values in a column surpasses a predefined threshold (e.g., threshold = 10000), the entire column is removed. This is suitable when a large number of odd values in a column suggests inconsistencies or issues with the data itself.
2. Row-Wise Removal (Optional)

Frequency Check (Optional): If column removal isn't applied or there are still odd values you want to address, consider this additional step.
Filtering by Row Odd Value Frequency: If desired, analyze rows and remove entire rows based on a threshold for the number of odd values they contain (e.g., remove rows with more than n odd values).



In [ ]:
dict = {}
for col in df_b.columns:
  count = (df_b[col] == -99999).sum()
  dict[col] = count

dict

{'PROSPECTID': 0,
 'time_since_recent_payment': 4291,
 'time_since_first_deliquency': 35949,
 'time_since_recent_deliquency': 35949,
 'num_times_delinquent': 0,
 'max_delinquency_level': 35949,
 'max_recent_level_of_deliq': 0,
 'num_deliq_6mts': 0,
 'num_deliq_12mts': 0,
 'num_deliq_6_12mts': 0,
 'max_deliq_6mts': 12890,
 'max_deliq_12mts': 10832,
 'num_times_30p_dpd': 0,
 'num_times_60p_dpd': 0,
 'num_std': 0,
 'num_std_6mts': 0,
 'num_std_12mts': 0,
 'num_sub': 0,
 'num_sub_6mts': 0,
 'num_sub_12mts': 0,
 'num_dbt': 0,
 'num_dbt_6mts': 0,
 'num_dbt_12mts': 0,
 'num_lss': 0,
 'num_lss_6mts': 0,
 'num_lss_12mts': 0,
 'recent_level_of_deliq': 0,
 'tot_enq': 6321,
 'CC_enq': 6321,
 'CC_enq_L6m': 6321,
 'CC_enq_L12m': 6321,
 'PL_enq': 6321,
 'PL_enq_L6m': 6321,
 'PL_enq_L12m': 6321,
 'time_since_recent_enq': 6321,
 'enq_L12m': 6321,
 'enq_L6m': 6321,
 'enq_L3m': 6321,
 'MARITALSTATUS': 0,
 'EDUCATION': 0,
 'AGE': 0,
 'GENDER': 0,
 'NETMONTHLYINCOME': 0,
 'Time_With_Curr_Empr': 0,
 'pct_of

In [ ]:
removed_keys = [key for key, value in dict.items() if value > 10000]

In [ ]:
removed_keys

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [ ]:
df_b.drop(removed_keys,axis=1,inplace = True)

# **Checking null values **

In [ ]:
df_b.isna().sum()

PROSPECTID                    0
time_since_recent_payment     0
num_times_delinquent          0
max_recent_level_of_deliq     0
num_deliq_6mts                0
num_deliq_12mts               0
num_deliq_6_12mts             0
num_times_30p_dpd             0
num_times_60p_dpd             0
num_std                       0
num_std_6mts                  0
num_std_12mts                 0
num_sub                       0
num_sub_6mts                  0
num_sub_12mts                 0
num_dbt                       0
num_dbt_6mts                  0
num_dbt_12mts                 0
num_lss                       0
num_lss_6mts                  0
num_lss_12mts                 0
recent_level_of_deliq         0
tot_enq                       0
CC_enq                        0
CC_enq_L6m                    0
CC_enq_L12m                   0
PL_enq                        0
PL_enq_L6m                    0
PL_enq_L12m                   0
time_since_recent_enq         0
enq_L12m                      0
enq_L6m 

# **Merging Datasets**: df_b and df_c

This section describes the process of merging two datasets, df_b and df_c, into a single dataset based on a common column. Merging allows you to combine information from different sources that share a common identifier.



**Common Column Selection**

The first step is to identify the column that exists in both datasets and will be used for the merge. This column ensures proper alignment of rows during the merge process.

In [ ]:
for i in list(df_b.columns):
    if i in list(df_c.columns):
        print(i)

PROSPECTID


In [ ]:
data = pd.merge(df_b,df_c,how = 'inner',left_on =['PROSPECTID'],right_on =['PROSPECTID'])

In [ ]:
data.head(2)

,PROSPECTID,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,549,11,29,0,0,0,0,0,21,...,0,0,1,0,4,1,4,0,72,18
1,2,47,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,7,7


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51296 entries, 0 to 51295
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  51296 non-null  int64  
 1   time_since_recent_payment   51296 non-null  int64  
 2   num_times_delinquent        51296 non-null  int64  
 3   max_recent_level_of_deliq   51296 non-null  int64  
 4   num_deliq_6mts              51296 non-null  int64  
 5   num_deliq_12mts             51296 non-null  int64  
 6   num_deliq_6_12mts           51296 non-null  int64  
 7   num_times_30p_dpd           51296 non-null  int64  
 8   num_times_60p_dpd           51296 non-null  int64  
 9   num_std                     51296 non-null  int64  
 10  num_std_6mts                51296 non-null  int64  
 11  num_std_12mts               51296 non-null  int64  
 12  num_sub                     51296 non-null  int64  
 13  num_sub_6mts                512

# **Feature Engineering Categorical Columns**

In [ ]:
categorical_variables=[col for col in data if data[col].dtype =='object']

In [ ]:
len(categorical_variables)

6

In [ ]:
categorical_variables

['MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

In [ ]:
for i in ['MARITALSTATUS', 'EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']:
    chi2,pval,_,_ =chi2_contingency(pd.crosstab(data[i],data['Approved_Flag']))
    print(i,'---',pval)

MARITALSTATUS --- 2.7347247145640217e-257
EDUCATION --- 6.772122013692765e-38
GENDER --- 0.000259241464445085
last_prod_enq2 --- 0.0
first_prod_enq2 --- 0.0


**Since all the categorical features have the pvalue<0.05, we can accept all the features**

# **Feature Engineering Numerical Columns**

In [ ]:
numeric_columns = []
for i in data.columns:
    if data[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)

In [ ]:
len(numeric_columns)

72

In [ ]:
numeric_columns

['time_since_recent_payment',
 'num_times_delinquent',
 'max_recent_level_of_deliq',
 'num_deliq_6mts',
 'num_deliq_12mts',
 'num_deliq_6_12mts',
 'num_times_30p_dpd',
 'num_times_60p_dpd',
 'num_std',
 'num_std_6mts',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_6mts',
 'num_dbt_12mts',
 'num_lss',
 'num_lss_6mts',
 'num_lss_12mts',
 'recent_level_of_deliq',
 'tot_enq',
 'CC_enq',
 'CC_enq_L6m',
 'CC_enq_L12m',
 'PL_enq',
 'PL_enq_L6m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L12m',
 'enq_L6m',
 'enq_L3m',
 'AGE',
 'NETMONTHLYINCOME',
 'Time_With_Curr_Empr',
 'pct_of_active_TLs_ever',
 'pct_opened_TLs_L6m_of_L12m',
 'pct_currentBal_all_TL',
 'CC_Flag',
 'PL_Flag',
 'pct_PL_enq_L6m_of_L12m',
 'pct_CC_enq_L6m_of_L12m',
 'pct_PL_enq_L6m_of_ever',
 'pct_CC_enq_L6m_of_ever',
 'HL_Flag',
 'GL_Flag',
 'Credit_Score',
 'Total_TL',
 'Tot_Closed_TL',
 'Tot_Active_TL',
 'Total_TL_opened_L6M',
 'Tot_TL_closed_L6M',
 'pct_tl_open_L6M',
 'pct_tl_clo

In [ ]:
vif_data = data[numeric_columns]
total_columns = vif_data.shape[1]
column_to_be_kept =[]
column_index=0

In [ ]:
vif_data.head(5)

,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,num_std_6mts,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,549,11,29,0,0,0,0,0,21,5,...,0,0,1,0,4,1,4,0,72,18
1,47,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,7,7
2,302,9,25,1,9,8,0,0,10,5,...,0,6,1,0,0,2,6,0,47,2
3,-99999,0,0,0,0,0,0,0,5,4,...,0,0,0,0,0,0,1,1,5,5
4,583,0,0,0,0,0,0,0,53,4,...,0,0,0,0,0,3,0,2,131,32


In [ ]:
for i in range (0,total_columns):

    vif_value = variance_inflation_factor(vif_data, column_index)
    print (column_index,'---',vif_value)


    if vif_value <= 6:
        column_to_be_kept.append(numeric_columns[i] )
        column_index = column_index+1

    else:
        vif_data = vif_data.drop([ numeric_columns[i] ] , axis=1)

0 --- 1.230044745024942
1 --- 7.197918487528533
1 --- 5.083224958714561
2 --- inf
2 --- 6.637030784474982
2 --- 1.3279631076357916
3 --- 8.08196100117126
3 --- 1.601495639502701
4 --- 6.105779885767202
4 --- 13.38644793135201
4 --- 2.2391414642912526
5 --- 1.5573667430744673
6 --- 2.5445702808113517
7 --- 3.0995414819629827
8 --- 2.19230444317388
9 --- 7.383731316308144
9 --- 2.0746495110640333
10 --- 2.7258502218753535
11 --- 6.292245618691434
11 --- 2.7102905199951066
12 --- 4.6868809405042375
13 --- 375963513.2180785
13 --- 2636138993.40204
13 --- 10959916643.130999
13 --- 1349003097.9418674
13 --- 1602016367.8343487
13 --- 5166295901.550022
13 --- 604830942.8305376
13 --- 8464.155860246026
13 --- 628687130.0422455
13 --- 735751972.5869627
13 --- 1.1816346570600254
14 --- 1.3983548793028717
15 --- 1.0498572051589397
16 --- 1.264218241731676
17 --- inf
17 --- 5.308118678274577
18 --- 1.0057773785142932
19 --- 2.8092237601993855
20 --- 2.313223854595055
21 --- 16.82641072071497
21 ---

In [ ]:
columns_to_be_kept_numerical = []

In [ ]:
from scipy.stats import f_oneway

In [ ]:
for i in column_to_be_kept:
    a = list(data[i])
    b = list(data['Approved_Flag'])

    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']


    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)


In [ ]:
len(columns_to_be_kept_numerical)

37

Significantly reduced the number of features in a dataset from 72 to 37

# **listing all the final features**

In [ ]:
features = columns_to_be_kept_numerical + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
data = data[features + ['Approved_Flag']]

# **Encoding Categorical Columns**

In [ ]:
print(data['MARITALSTATUS'].unique())
print('----'*20)
print(data['EDUCATION'].unique())
print('----'*20)
print(data['GENDER'].unique())
print('----'*20)
print(data['last_prod_enq2'].unique())
print('----'*20)
print(data['first_prod_enq2'].unique())

['Married' 'Single']
--------------------------------------------------------------------------------
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
--------------------------------------------------------------------------------
['M' 'F']
--------------------------------------------------------------------------------
['PL' 'ConsumerLoan' 'others' 'AL' 'CC' 'HL']
--------------------------------------------------------------------------------
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


In [ ]:
data.loc[data['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
data.loc[data['EDUCATION'] == '12TH',['EDUCATION']]             = 2
data.loc[data['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 3
data.loc[data['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
data.loc[data['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 4
data.loc[data['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
data.loc[data['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 3

In [ ]:
data['EDUCATION'].value_counts()
data['EDUCATION'] = data['EDUCATION'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51296 entries, 0 to 51295
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   time_since_recent_payment   51296 non-null  int64  
 1   max_recent_level_of_deliq   51296 non-null  int64  
 2   num_deliq_6_12mts           51296 non-null  int64  
 3   num_times_60p_dpd           51296 non-null  int64  
 4   num_std_12mts               51296 non-null  int64  
 5   num_sub                     51296 non-null  int64  
 6   num_sub_12mts               51296 non-null  int64  
 7   num_dbt                     51296 non-null  int64  
 8   num_dbt_12mts               51296 non-null  int64  
 9   num_lss                     51296 non-null  int64  
 10  recent_level_of_deliq       51296 non-null  int64  
 11  enq_L3m                     51296 non-null  int64  
 12  AGE                         51296 non-null  int64  
 13  NETMONTHLYINCOME            512

In [ ]:
df_encoded = pd.get_dummies(data, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])

In [ ]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51296 entries, 0 to 51295
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   time_since_recent_payment     51296 non-null  int64  
 1   max_recent_level_of_deliq     51296 non-null  int64  
 2   num_deliq_6_12mts             51296 non-null  int64  
 3   num_times_60p_dpd             51296 non-null  int64  
 4   num_std_12mts                 51296 non-null  int64  
 5   num_sub                       51296 non-null  int64  
 6   num_sub_12mts                 51296 non-null  int64  
 7   num_dbt                       51296 non-null  int64  
 8   num_dbt_12mts                 51296 non-null  int64  
 9   num_lss                       51296 non-null  int64  
 10  recent_level_of_deliq         51296 non-null  int64  
 11  enq_L3m                       51296 non-null  int64  
 12  AGE                           51296 non-null  int64  
 13  N

# **Model**

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)

y = df_encoded['Approved_Flag']
x = df_encoded.drop( ['Approved_Flag'], axis = 1 )


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)



# **Model Accuracy**

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy:.2f}')
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.99

Class p1:
Precision: 0.9857270294380018
Recall: 0.9650655021834061
F1 Score: 0.9752868490732568

Class p2:
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Class p3:
Precision: 0.9732441471571907
Recall: 0.9891230455472467
F1 Score: 0.9811193526635199

Class p4:
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

